In [11]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

## Ravnotežne točke kot funkcija vhoda v sistem in modalnih oblik sistema

Ta interaktivni primer prikazuje ravnotežne točke linearnih časovno nespremenljivih (LTI) sistemov. Namen primera je pokazati, kje ležijo ravnotežne točke v prostoru stanj (v tem konkretnem primeru v 2D ravnini), kot funkcija matrik sistema in vhoda v sistem.

Dinamiko LTi sistema popišemo kot:

$$
\dot{x}(t)=Ax(t)+B\bar{u},
$$

kjer je $\bar{u}$ konstantna vhodna funkcija v sistem. Ravnotežne točke sistema določimo z rešitvijo naslednje enačbe:

$$
Ax(t)=-B\bar{u}.
$$


### Kako upravljati s tem interaktivnim primerom?

Poizkusi določiti matrike sistema in vhod v sistem $\bar u$ tako, da dosežeš naslednje:
* sistem ima eno ravnotežno točko v izhodišču,
* sistem ima eno ravnotežno točko, ki pa ne leži v izhodišču,
* sistem ima $\infty$ ravnotežnih točk,
* sistem ima $\infty^2$ ravnotežnih točk.

In [12]:
#Preparatory Cell 

import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#create a NxM matrix widget 
def createMatrixWidget(n,m):
    M = widgets.GridBox(children=[widgets.FloatText(layout=widgets.Layout(width='100px', height='40px'),
    value=0.0, disabled=False, label=i) for i in range(n*m)],
    layout=widgets.Layout(
        #width='50%',
        grid_template_columns= ''.join(['100px ' for i in range(m)]),
        #grid_template_rows='80px 80px 80px',
        grid_row_gap='0px',
        track_size='0px')
    )
    return M


#extract matrix from widgets and convert to numpy matrix
def getNumpyMatFromWidget(M,n,m):
    #get W gridbox dims
    M_ = numpy.matrix(numpy.zeros((n,m)))
    for irow in range(0,n):
        for icol in range(0,m):
            M_[irow,icol] = M.children[irow*3+icol].value

            
#this is a simple derived class from FloatText used to experience with interact             
class floatWidget(widgets.FloatText):
    def __init__(self,**kwargs):
        #self.n = n
        self.value = 30.0
        #self.M = 
        widgets.FloatText.__init__(self, **kwargs)

#    def value(self):
#        return 0 #self.FloatText.value

from traitlets import Unicode
from ipywidgets import register 


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

        

        
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass



In [13]:
#define matrices
A = matrixWidget(2,2)
B = matrixWidget(2,1)
ubar = matrixWidget(1,1)

A.setM(-numpy.identity(2))

#this is the main callback and does all the computations and plots 
def main_callback(matA,matB,ubar_,DW,sel):
    #check if a specific matrix is requested or is manual 
    if sel== 'ročna nastavitev':
        pass
    elif sel == 'ena ravnotežna točka':
        matA = numpy.zeros((2,2))
        matA[0,0] = -1
        matA[1,1] = -2
        A.setM(matA)
        matB = numpy.ones((2,1))
        B.setM(matB)
        ubar_ = numpy.ones((1,1))
        ubar.setM(ubar_)
    elif sel == 'neskončno ravnotežnih točk':
        matA = numpy.zeros((2,2))
        matA[0,0] = -1
        matA[0,1] = 2
        A.setM(matA)
        matB = numpy.zeros((2,1))
        B.setM(matB)
        ubar_ = numpy.zeros((1,1))
        ubar.setM(ubar_)
    elif sel == 'neskončno^2 ravnotežnih točk':
        matA = numpy.zeros((2,2))
        A.setM(matA)
        matB = numpy.zeros((2,1))
        B.setM(matB)
        ubar_ = numpy.zeros((1,1))
        ubar.setM(ubar_)
    else : 
        matA = numpy.zeros((2,2))
        matA[0,0] = -1
        matA[1,1] = -1
        A.setM(matA)
        matB = numpy.zeros((2,1))
        B.setM(matB)
        ubar_ = numpy.zeros((1,1))
        ubar.setM(ubar_)
        
        
        
    #get system eigenvalues
    lambdas, eigvectors = numpy.linalg.eig(matA)

    #count eigenvalues in 0
    #NOTE: when extracting i-th right eignvectors (columns from the matrix eigvectors) 
    # must use the notation : eigvectors[:,i:i+1] that generates a column array 
    # because the notation eigvectors[:,i] generates a row array !!!! WTF MBM
    eigin0 = 0    
    if lambdas[0] == 0.0:
        eig0 = True
        dir0 = eigvectors[:,0:1]
        eigin0 = eigin0+1
    else:
        eig0 = False
    if lambdas[1] == 0.0:
        eig1 = True
        dir1 = eigvectors[:,1:2]
        eigin0 = eigin0+1
    else: 
        eig1 = False

    #create textual output            
    display(Markdown('Matrika $%s$ ima $%s$ lastni(h) vrednosti enaki(h) 0.' % (vmatrix(matA), str(eigin0)) ))
    #create modes string:

    
    
    #test if aoslution exist: 
    if eigin0 == 0:
        #a solution always exists
        print('Sistem ima le eno ravnotežno točko.')
        pass
    else:
        # in order for a solution of Ax = -Bu, with A singular, it is necessary that
        # rank([A b])=rank(A)
        # if this does not happen pinv cannot be used !!!
        if numpy.linalg.matrix_rank(numpy.concatenate((matA,matB), axis = 1)) == numpy.linalg.matrix_rank(matA):
            #a solution exists
            if eigin0 == 1:
                print('Sistem ima neskončno ravnotežnih točk, ki ležijo na premici.')
            else:
                print('Sistem ima neskončno^2 ravnotežnih točk, ki zasedajo celoten prostor stanj.')
        else:
            #a solution does not exist 
            print('Opozorilo: sistem nima rešitve in s tem nobene ravnotežne točke.')
            return
   
    #print(lambdas)
    #print('----')
    #print(eigvectors)
    #print(eigvectors[:,1:2])
    #v1 = eigvectors[:,]
    #print(eigvectors.__type__)
    #print(v1.__type__)
    #print(eigvectors[:,0:1])
    #print('----')
    
    #compute equilibrium points 
    if eigin0 == 0 : 
        #only one equilibrium point 
        eq = - numpy.dot(numpy.linalg.inv(matA),matB)*ubar_
        eqdir = numpy.zeros((2,1))
    elif eigin0 == 1:
        #equilibrium along a line 
        eq = - numpy.dot(numpy.linalg.pinv(matA),matB)*ubar_
        if eig0:
            eqdir = dir0
        else:
            eqdir = dir1
    else:
        #equilibrium is the entire plane
        eq = numpy.zeros((2,1))
        eqdir = numpy.zeros((2,1))
        pass
    
    #set limits of plot 
    xlim = max(abs(eq[0,0])*1.1, 1.)
    ylim = max(abs(eq[1,0])*1.1, 1.)

    #print(eq)
    #print(eqdir)
    #print(xlim)
    #print(ylim)
    
    #plot equilibrium points (poles only)    
    pzmap = plt.figure(figsize=(6,6))
    sf = pzmap.add_subplot(111)
    sf.grid(True)
    sf.set_xlabel('$x_1$')
    sf.set_ylabel('$x_2$')
    sf.set_xlim([-xlim, xlim])
    sf.set_ylim([-ylim, ylim])
    #sf.set_aspect('equal', adjustable='datalim')
    if eigin0 == 0:
        #one equilibrium point
        sf.plot(eq[0,0],eq[1,0],marker='o')
    elif eigin0 == 1:
        #infinite equilibrium points along a line
        sf.plot(eq[0,0],eq[1,0],marker='o')
        #ph.hold(True)
        sf.plot([eq[0,0]-eqdir[0,0]*xlim*10, eq[0,0]+eqdir[0,0]*xlim*10],[eq[1,0]-eqdir[1,0]*ylim*10, eq[1,0]+eqdir[1,0]*ylim*10])
    else:
        #infinite^2 equilibrium points occupying the entire state plane 
        sf.fill((-xlim, xlim, xlim, -xlim),(-ylim, -ylim, ylim, ylim),alpha = 0.5)
            
            
   
    
#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

#define type of ipout 
SELECT = widgets.Dropdown(
    options=['ročna nastavitev', 'ponastavi', 'ena ravnotežna točka', 
             'neskončno ravnotežnih točk', 
             'neskončno^2 ravnotežnih točk'],
    value='ročna nastavitev',
    description='Primeri:',
    disabled=False,
)


#create a graphic structure to hold all widgets 
alltogether =  widgets.VBox([widgets.HBox([widgets.Label('$\dot{x}(t) = $',border=3), A,widgets.Label('$x(t) + $',border=3),B,widgets.Label('$\\bar{u}$',border=3),ubar, START]),
                             SELECT ] )
    

out = widgets.interactive_output(main_callback,{'matA': A, 'matB': B,'ubar_': ubar,'DW': DW, 'sel': SELECT})
display(alltogether,out)



Output()